# 2017-05-02 : grabbing the framerate and plotting it
----------------------------

- [x] Capturing a framerate for each downscale parameter possible

Obtaining different framerates will allow us to chose which downscale using for an optimum recording.  
The framerate seems to be hightly dependent of the program simplicity. Going from : 

    (...)
    for i in range(0,num_frames):
        Ph.grab()
        frame = Ph.grab()
        (...)
        
to :

    (...)
    for i in range(0,num_frames):
        frame = Ph.grab()
        (...)
        
Seem to double the framerate recorded (from 7 to 13 FPS). It is understandable since the first version executed Ph.grab() two times per loop. Also, the version b (limit : time recorded) seems to be a bit faster than the version a (limit : number of frames recorded) as it record at 14-15 FPS (vs 13 FPS).  
The program contained a little writing error that prevented it to work correctly when the downscale parameter was superior to 1. Going from :

    (...)
    if DOWNSCALE > 1 :
        W = self.cap.get(cv2.cv.CV_CAP_PROP_FRAME_WIDTH)
        H = self.cap.set(cv2.cv.CV_CAP_PROP_FRAME_HEIGHT)
        (...)

to :

    (...)
    if DOWNSCALE > 1 :
        W = self.cap.get(cv2.cv.CV_CAP_PROP_FRAME_WIDTH)
        H = self.cap.get(cv2.cv.CV_CAP_PROP_FRAME_HEIGHT)
        (...)    

I built a loop that allow to modify the downscale parameter :

    (...)
    for ds in range(10):
        Ph = PhotoReceptor(w=1280, h=720, DOWNSCALE = ds)
        (...)

In these conditions, the framerate seems to not be influenced by the downscale, as the program output the same FPS for each of them (except for downscale = 2).

    Before downscale. dim1 : 720, dim2 : 1280
    Using OpenCV
    After downscale 1. dim1 : 720.0, dim2 : 1280.0
    Frame rate : 15.0 

    Before downscale. dim1 : 720, dim2 : 1280
    Using OpenCV
    After downscale 2. dim1 : 288.0, dim2 : 352.0
    Frame rate : 17.0 

    Before downscale. dim1 : 720, dim2 : 1280
    Using OpenCV
    After downscale 3. dim1 : 240.0, dim2 : 320.0
    Frame rate : 15.0 

    Before downscale. dim1 : 720, dim2 : 1280
    Using OpenCV
    After downscale 4. dim1 : 144.0, dim2 : 176.0
    Frame rate : 15.0
    
    (...)
    
In addition, a problem appear from downscale = 7 to beyond : the new dimensions are not reduced as expected. It looks like the program is not able to reduce them this much and so it may represent the upper limit of the downscaling in these conditions.  

    (...)
    
    Before downscale. dim1 : 720, dim2 : 1280
    Using OpenCV
    After downscale 5. dim1 : 144.0, dim2 : 176.0
    Frame rate : 15.0 

    Before downscale. dim1 : 720, dim2 : 1280
    Using OpenCV
    After downscale 6. dim1 : 120.0, dim2 : 160.0
    Frame rate : 15.0 

    Before downscale. dim1 : 720, dim2 : 1280
    Using OpenCV
    After downscale 7. dim1 : 480.0, dim2 : 640.0
    Frame rate : 15.0 

    Before downscale. dim1 : 720, dim2 : 1280
    Using OpenCV
    After downscale 8. dim1 : 480.0, dim2 : 640.0
    Frame rate : 15.0 
    
    (...)
    
More informations about plotting with matplotlib : https://matplotlib.org/users/pyplot_tutorial.html   
More informations about plot() : https://matplotlib.org/api/pyplot_api.html#matplotlib.pyplot.plot

-  [x] Building a plot comparing the framerate with the downscale parameter

The plot construction is simple. It introduce two lists that get completed during the program execution :

    (...)
    downscales = []
    frames = []
    (...)
        frames.append(fps)
        downscales.append(ds)
        (...)
        
That get called by a few lines :

    plt.plot(downscales, frames)
    plt.title('Effect of downscale on framerate')
    plt.xlabel('Downscale')
    plt.ylabel('FPS')
    plt.show()

The program correctly shows the plot but it doesn't give any useful information, because the framerate doesn't seems influenced by the downscale.  
I tried to obtain more detailed informations, so I modified a few lines. Going from : 

    (...)
    for ds in range(1,7):
        (...)
        fps = nb_frames // seconds
        (...)
        
to :

    (...)
    for ds in np.arrange(1,7,0.5): #Modification needed because range() doesn't use floats, but np.arange() do
        (...)
        fps = nb_frames / seconds
        (...)
        
After some tries, it seems that the framerate is hightly test variable but it almost not influenced by the downscale.
        
I need to check with L. Perrinet if everything in the program is alright before going on the next steps.  

# 2017-05-03
------------------------

I modified the program to integrate the framerate grabbing into the grab() function :

    def grab(self, nb_frames) :
            self.start = time.time()

            for nb in range(nb_frames):
                ret, frame_bgr = self.cap.read()
                #frame = frame_bgr[:, :, ::-1] #BGR to RBG.
                frame = frame_bgr

                self.end = time.time()
                self.seconds = self.end - self.start

                fps = nb_frames / self.seconds
                print ('Framerate :', fps,'\n')

                return frame, fps
                
That allowed me to make the calling block simplier :

    nb_frames = 50
    downscales_a, frames_a = [], []
    for ds in np.arange(1,7,0.5):

        Ph = PhotoReceptor(w=1280, h=720, DOWNSCALE = ds)

        frame, fps = Ph.grab(nb_frames)

        Ph.close()

        frames_a.append(fps)
        downscales_a.append(ds)
    
But the difference seems to have unbelievably speed up the framerate, going from about 16 FPS to more than 70 FPS. There must be an error in the framerate computation. Returning to the last checkpoint.

- [ ] Etablishing a communication between the program and a distant camera

The communication must contain 3 things :  
1) A way to ask the camera if it's ready to record  
2) A way for the camera to respond positively or negatively  
3) A way to cut down the communication  

The communication can be etablished through a program that'll look like this pseudocode :

    __init__
    while True:
        grab()
        code
        send/recieve
    close()
    
More informations about the picamera library, allowing to communicate with a camera connected to a Raspberry Pi : https://github.com/laurentperrinet/openRetina/blob/master/src/openRetina.py  
Complete documentation : http://picamera.readthedocs.io/en/release-1.13/  
Note that may be important if recording a video or using a long exposure time : the picamera use a rolling shutter (vs global shutter, more informations : https://en.wikipedia.org/wiki/Rolling_shutter)  
A lot of informations about the current objective is available here : http://picamera.readthedocs.io/en/release-1.13/recipes1.html#capturing-to-a-network-stream  

The program must be divided in two scripts : one on the RaspBerry Pi that'll be used as a client, the other on a computer that'll be used as a server.  
The client will first send a 32-bit integer containing the length of the image that'll follow. If this length is 0, this indicate that no more images will be sent and the connection is shuted down.

    connection = server_socket.accept()[0].makefile('rb')
    
A problem seem to be provoked by the previous codeline. The program seem unable to go beyond it.